In [3]:
import os
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
import pandas as pd

API_KEY = 'your_api_key_here'
STOCK_TO_TRACK = 'BTC'
CASH = 111

def get_stock_data(symbol, interval='daily', outputsize='compact'):
    ts = TimeSeries(key=API_KEY, output_format='pandas')
    data, _ = ts.get_daily_adjusted(symbol=symbol, outputsize=outputsize)
    return data

def get_moving_averages(symbol, interval='daily', time_period=20, series_type='close'):
    ti = TechIndicators(key=API_KEY, output_format='pandas')
    data, _ = ti.get_sma(symbol=symbol, interval=interval, time_period=time_period, series_type=series_type)
    return data

def execute_trading_strategy():
    stock_data = get_stock_data(STOCK_TO_TRACK)
    sma_20 = get_moving_averages(STOCK_TO_TRACK, time_period=20)
    sma_50 = get_moving_averages(STOCK_TO_TRACK, time_period=50)

    merged_data = pd.concat([stock_data, sma_20, sma_50], axis=1).dropna()
    merged_data.columns = ['open', 'high', 'low', 'close', 'adjusted_close', 'volume', 'dividend', 'split_coeff', 'sma_20', 'sma_50']

    in_position = False
    shares = 0
    profit = 0

    for index, row in merged_data.iterrows():
        if not in_position and row['sma_20'] > row['sma_50']:
            shares = CASH // row['adjusted_close']
            in_position = True
            print(f"Buying {shares} shares of {STOCK_TO_TRACK} at {row['adjusted_close']}")
        elif in_position and row['sma_20'] < row['sma_50']:
            in_position = False
            profit += shares * row['adjusted_close'] - CASH
            print(f"Selling {shares} shares of {STOCK_TO_TRACK} at {row['adjusted_close']}, profit: {profit}")
            break

if __name__ == "__main__":
    execute_trading_strategy()


ValueError: Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_DAILY_ADJUSTED.